In [4]:
import tensorflow as tf
from tensorflow import keras

In [5]:
print(tf.__version__)
print(keras.__version__)

1.12.0
2.1.6-tf


## Import the dataset

In [6]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
# train_targets are one-hot encodingss
train_files, train_targets = load_dataset('dog-images-training/train')
valid_files, valid_targets = load_dataset('dog-images-training/valid')
test_files, test_targets = load_dataset('dog-images-training//test')

dog_names = np.load('./labels/dog_names.npy')

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files)) 

There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


## Preprocessing functions

In [7]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

## Prediction

### Prediction function

In [8]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

# Input: Imagepath
# Output: string of dogname
def dog_predictor(model, img_path):
    img = preprocess_input(path_to_tensor(img_path))
    prediction = model(img_path)
    return dog_names[np.argmax(ResNet50_model.predict(img))]

## Developing the model

In [9]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files[:1000]).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

print("The shape of input: ", train_tensors[0].shape)

100%|██████████| 836/836 [00:06<00:00, 127.92it/s]


The shape of input:  (224, 224, 3)


### Define the model

In [10]:
from keras.applications import VGG19, ResNet50
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 

img_width, img_height = 224, 224

model = ResNet50(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

# Freeze the layers which you don't want to train
for layer in model.layers:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(len(dog_names), activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)
model_final.summary()

/home/akshata/.local/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

/home/akshata/.conda/envs/dog-project/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


### Compile the model

In [12]:
from keras import optimizers

model_final.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [13]:
from keras.callbacks import ModelCheckpoint  

epochs=2
checkpointer = ModelCheckpoint(filepath='keras_models/weights.best_single_model.hdf5', 
                               verbose=1, save_best_only=True)

model_final.fit(train_tensors, train_targets[:1000], 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 1000 samples, validate on 835 samples
Epoch 1/5
1000/1000 [==============================] - 384s 384ms/step - loss: 15.7927 - acc: 0.0080 - val_loss: 16.0023 - val_acc: 0.0072

Epoch 00001: val_loss improved from inf to 16.00228, saving model to keras_models/weights.best_single_model.hdf5
Epoch 2/5
1000/1000 [==============================] - 299s 299ms/step - loss: 15.9408 - acc: 0.0110 - val_loss: 16.0023 - val_acc: 0.0072

Epoch 00002: val_loss did not improve from 16.00228
Epoch 3/5
1000/1000 [==============================] - 285s 285ms/step - loss: 15.9408 - acc: 0.0110 - val_loss: 16.0023 - val_acc: 0.0072

Epoch 00003: val_loss did not improve from 16.00228
Epoch 4/5
1000/1000 [==============================] - 286s 286ms/step - loss: 15.9408 - acc: 0.0110 - val_loss: 16.0023 - val_acc: 0.0072

Epoch 00004: val_loss did not improve from 16.00228
Epoch 5/5
1000/1000 [==============================] - 286s 286ms/step - loss: 15.9408 - acc: 0.0110 - val_loss: 16.0023 - v

### Test accuracy of the model

In [16]:
model_final.load_weights('keras_models/weights.best_single_model.hdf5')


# get index of predicted dog breed for each image in test set
dog_breed_predictions = [ np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors ]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)

## Test Model Predictions